In [2]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('Dataset/Dataset_04.csv')

In [3]:
dataset

,LOCATION,SUBJECT,TIME,Value
0,AUS,BEEF,1991,27.721815
1,AUS,BEEF,1992,26.199591
2,AUS,BEEF,1993,26.169094
3,AUS,BEEF,1994,25.456134
4,AUS,BEEF,1995,25.340226
...,...,...,...,...
6713,EU28,SHEEP,2022,2.012270
6714,EU28,SHEEP,2023,2.018727
6715,EU28,SHEEP,2024,2.026332
6716,EU28,SHEEP,2025,2.034105


In [4]:
# 첫번째 문제
q1 = dataset.copy()

In [12]:
q1.columns

Index(['LOCATION', 'SUBJECT', 'TIME', 'Value'], dtype='object')

In [6]:
q1 = q1[q1['LOCATION'] == "KOR"]

In [14]:
# 년도별 육류 소비량 합계를 구하기
q1_out=pd.pivot_table(data=q1, index='TIME', values='Value',aggfunc='sum')
q1_out

,Value
TIME,
1990,5.774255
1991,21.911207
1992,26.901769
1993,26.938255
1994,27.877341
1995,30.042035
1996,31.928922
1997,32.002991
1998,30.009057


In [15]:
q1_out.reset_index(inplace=True)

In [21]:
# 상관관계 분석
q1_out.corr()['Value']['TIME'].round(3)

0.96

---

In [22]:
# 두번째 문제
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel


In [23]:
q2 = dataset.copy()

In [24]:
q2 = q2[q2.LOCATION.isin(['KOR','JPN'])]
q2

,LOCATION,SUBJECT,TIME,Value
294,JPN,BEEF,1990,6.111035
295,JPN,BEEF,1991,6.517055
296,JPN,BEEF,1992,6.766244
297,JPN,BEEF,1993,7.406407
298,JPN,BEEF,1994,8.129090
...,...,...,...,...
583,KOR,SHEEP,2022,0.197951
584,KOR,SHEEP,2023,0.199098
585,KOR,SHEEP,2024,0.200266
586,KOR,SHEEP,2025,0.201451


In [25]:
q2_out = pd.pivot_table(data=q2, index=['TIME','SUBJECT'],
                       columns=['LOCATION'],
                        values='Value')

In [27]:
# 결측치 처리
q2_out=q2_out.dropna()

q2_out=q2_out.reset_index()
q2_out

LOCATION,TIME,SUBJECT,JPN,KOR
0,1990,POULTRY,12.844865,5.570102
1,1990,SHEEP,0.756696,0.204153
2,1991,BEEF,6.517055,5.085481
3,1991,PIG,13.239267,9.930182
4,1991,POULTRY,12.055422,6.738203
...,...,...,...,...
141,2025,SHEEP,0.125060,0.201451
142,2026,BEEF,6.844875,10.737425
143,2026,PIG,15.544018,29.544063
144,2026,POULTRY,14.768340,17.907865


In [29]:
sub_list = q2_out.SUBJECT.unique()
# array(['POULTRY', 'SHEEP', 'BEEF', 'PIG'], dtype=object)

In [30]:
from scipy.stats import ttest_rel
q2_out2 = []

for i in sub_list:
    temp = q2_out[q2_out.SUBJECT == i]
    pvalue = ttest_rel(temp['KOR'], temp['JPN']).pvalue
    q2_out2 = q2_out2 + [[i, pvalue]]
    
q2_out2=pd.DataFrame(q2_out2, columns=['sub','pvalue'])

In [32]:
# 차이가 없다는 말은 귀무가설을 채택하겠다는 의미이므로 
# pvalue가 0.05이상인 경우 귀무가설 기각하지 않음, 즉 채택

q2_out2[q2_out2.pvalue >= 0.05]
# 정답 : POULTRY

,sub,pvalue
0,POULTRY,0.469063


---

In [36]:
# 세번째 문제
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS, add_constant
from sklearn.metrics import r2_score, mean_absolute_error # 결정계수

q3 = q2_out.drop(columns='JPN')

q3['TIME'].shape #(146,)
q3[['TIME']].shape #(146,1)

q3_out=[]
sub_list = q3.SUBJECT.unique()

for i in sub_list:
    temp = q3[q3.SUBJECT == i]
    lm = LinearRegression().fit(temp[['TIME']], temp.KOR)
    # X(입력변수) 2차원 구조로 입력
    r2_score = lm.score(temp[['TIME']], temp.KOR)
    q3_out = q3_out + [[i, r2_score]]
    
q3_out = pd.DataFrame(q3_out, columns=['sub','r2_score'])


In [39]:
temp = q3[q3.SUBJECT == 'POULTRY']
lm2 = LinearRegression().fit(temp[['TIME']], temp.KOR)
pred = lm2.predict(temp[['TIME']])

mape = (abs(temp.KOR - pred)/temp.KOR).sum() * 100/len(temp)
print(mape.round(3))
# 정답 : 5.783s

5.783


---

## (기타) 확장

In [40]:
q3_out=[]
sub_list = q3.SUBJECT.unique()

for i in sub_list:
    temp = q3[q3.SUBJECT == i]
    globals()['lm_' + str(i)]= LinearRegression().fit(temp[['TIME']], temp.KOR)
    # X(입력변수) 2차원 구조로 입력
    r2_score = eval('lm_' + str(i)).score(temp[['TIME']], temp.KOR)
    q3_out = q3_out + [[i, r2_score]]
    
q3_out = pd.DataFrame(q3_out, columns=['sub','r2_score'])


In [41]:
q3_out

,sub,r2_score
0,POULTRY,0.951498
1,SHEEP,0.004083
2,BEEF,0.835456
3,PIG,0.940681


In [43]:
lm_PIG.coef_

array([0.55942423])